In [1]:
import os
import sys
import argparse
import glob
import numpy as np
import pickle
import json
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
from projection_utils import Projection
from data_parser import read_keypoints

sys.path.insert(1, '../')
from virtual_huams_resource.utils import get_smplx_body_model

sys.path.insert(0, "../virtual_huams_resource")
from virtual_huams_resource.benji_prox_dataloader import DatasetBase, proxDatasetSkeleton

INFO - 2022-05-30 10:00:12,056 - acceleratesupport - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [2]:
data_root = '../N3OpenArea_00157_01/seq_0'
scene_dir = os.path.join("../quantitative", "scenes", "N3OpenArea.ply")

In [3]:
# define limbs
LIMBS = [(23, 15),
         (24, 15),
         (15, 22),
         (22, 12),
         # left arm
         (12, 13),
         (13, 16),
         (16, 18),
         (18, 20),
         # right arm
         (12, 14),
         (14, 17),
         (17, 19),
         (19, 21),
         # spline
         (12, 9),
         (9, 6),
         (6, 3),
         (3, 0),
         # left leg
         (0, 1),
         (1, 4),
         (4, 7),
         (7, 10),
         # right leg
         (0, 2),
         (2, 5),
         (5, 8),
         (8, 11)]
# define visualizer
vis = o3d.visualization.VisualizerWithKeyCallback()
# load point cloud
scene_point_cloud = o3d.io.read_point_cloud(os.path.join("../quantitative", "scenes", "N3OpenArea.ply"))

In [4]:
# get list of directories of the skeletons
ground_truth_path = "joints_worldnorm.pkl"
prediction_path = "transformer_predicted_joints_worldnorm.pkl"
ground_truth_dir_list = glob.glob(os.path.join(data_root, "s001_frame_00*", ground_truth_path))
ground_truth_dir_list.sort()
prediction_dir_list = glob.glob(os.path.join(data_root, "s001_frame_00*", prediction_path))
prediction_dir_list.sort()

In [5]:
import torch
import io

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

# get list of skeletons for ground truth and prediction
ground_truth_list = []      # list of skeletons of ground truth (dtype: numpy)
for dir in ground_truth_dir_list:
    with open(dir, 'rb') as f:
        data = pickle.load(f)
    data = data.detach().numpy()
    data = data.squeeze()
    data = data.astype(np.float64)
    ground_truth_list.append(data)

prediction_list = []      # list of skeletons of prediction (dtype: numpy)
for dir in prediction_dir_list:
    with open(dir, 'rb') as f:
        data = CPU_Unpickler(f).load()
    # data = torch.load(dir, map_location=torch.device('cpu'))
    data = data.detach().numpy()
    data = data.squeeze()
    data = data.astype(np.float64)
    prediction_list.append(data)

In [6]:
ground_truth_iter = iter(ground_truth_list)
prediction_iter = iter(prediction_list)
def update(vis):
    gt_skeleton = next(ground_truth_iter)          # render first frame
    pr_skeleton = next(prediction_iter)          # render first frame
    gt_skeleton_input = o3d.geometry.LineSet(
            o3d.utility.Vector3dVector(gt_skeleton),
            o3d.utility.Vector2iVector(LIMBS))
    gt_skeleton_input = gt_skeleton_input.paint_uniform_color([[0], [255], [0]])
    pr_skeleton_input = o3d.geometry.LineSet(
            o3d.utility.Vector3dVector(pr_skeleton),
            o3d.utility.Vector2iVector(LIMBS))
    pr_skeleton_input = pr_skeleton_input.paint_uniform_color([[255], [0], [0]])
    ctrl = vis.get_view_control()
    cam_param = ctrl.convert_to_pinhole_camera_parameters()
    vis.clear_geometries()
    vis.add_geometry(scene_point_cloud)
    vis.add_geometry(gt_skeleton_input)
    vis.add_geometry(pr_skeleton_input)
    ctrl.convert_from_pinhole_camera_parameters(cam_param)
    vis.poll_events()
    vis.update_renderer()
    vis.run()
    
# ______________ animate skeleton in point cloud ______________
vis.create_window()
vis.register_key_callback(65, update)       # press A
vis.add_geometry(scene_point_cloud)

gt_skeleton = ground_truth_list[0]
gt_skeleton_input = o3d.geometry.LineSet(
            o3d.utility.Vector3dVector(gt_skeleton),
            o3d.utility.Vector2iVector(LIMBS))
gt_skeleton_input = gt_skeleton_input.paint_uniform_color([[0], [255], [0]])
pr_skeleton = prediction_list[0]
pr_skeleton_input = o3d.geometry.LineSet(
            o3d.utility.Vector3dVector(pr_skeleton),
            o3d.utility.Vector2iVector(LIMBS))
pr_skeleton_input = pr_skeleton_input.paint_uniform_color([[255], [0], [0]])
vis.add_geometry(gt_skeleton_input)
vis.add_geometry(pr_skeleton_input)
vis.get_render_option().line_width = 1000
vis.run()
vis.destroy_window()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, cli

StopIteration: 

: 